In [1]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output


import albumentations as A
from albumentations.pytorch import ToTensorV2

import sklearn.metrics as metrics

import copy


num_workers = 3
initial_seed = 0
torch.manual_seed(initial_seed)

pretrained=True

base_path = "/int_data/house-count"

#train_alias = "train_a"
train_alias = "train"

In [2]:
#set reproducible executing
    
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    

In [3]:
if pretrained:
#    mean=(0.485, 0.456, 0.406)
#    std=(0.229, 0.224, 0.225)
    mean = (0.3945,  0.3966, 0.3185)
    std = (0.1600, 0.1648, 0.1746)
else:    
    mean = (0.3945,  0.3966, 0.3185)
    std = (0.1600, 0.1648, 0.1746)
    
resize_val = 256

transform_train = A.Compose([
        A.RandomRotate90(),
        A.Flip(),
        A.Transpose(),
        A.OneOf([
            A.ISONoise(),
            A.GaussNoise(),
        ], p=0.2),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        #A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),            
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),            
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
        A.Resize(resize_val, resize_val),
        A.Normalize(mean=mean, std=std),
        ToTensorV2()
    ])

transform_valid = A.Compose([        
        A.Resize(resize_val, resize_val),
        A.Normalize(mean=mean, std=std),
        ToTensorV2()
    ])

In [4]:
class ImageDataset(Dataset):
    def __init__(self, data_df, train_alias, transform=None):

        self.data_df = data_df
        self.transform = transform       

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name, label = self.data_df.iloc[idx]['img_num'], self.data_df.iloc[idx]['number_of_houses']

        # читаем картинку. read the image
        image = cv2.imread(f"{base_path}/{train_alias}/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
              
        if self.transform:
            image = self.transform(image=image)["image"]
        
        return image, torch.tensor(label).long()
    
    def __len__(self):
        return len(self.data_df)

In [5]:
test_df = pd.read_csv(base_path+"/sample_solution.csv")
test_df = test_df.drop(["number_of_houses"], axis = 1)


class TestImageDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        image_name = self.data_df.iloc[idx]['img_num']
        
        # читаем картинку
        image = cv2.imread(f"{base_path}/test/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)       
        
        # преобразуем, если нужно
        if self.transform:
            image = self.transform(image=image)["image"]
        
        return image
    
    def __len__(self):
        return len(self.data_df)        

In [6]:
def train(model, criterion, optimizer, scheduler, train_dataloader, test_dataloader, NUM_EPOCH=15):
    train_loss_log = []
    val_loss_log = []
    
    train_acc_log = []
    val_acc_log = []
    best_model_attr = None
    
    clear_output()
        
    
    for epoch in tqdm(range(NUM_EPOCH)):
        model.train()
        train_loss = 0.
        train_size = 0
        
        train_pred = 0.

        for imgs, labels in train_dataloader:
            optimizer.zero_grad()

            imgs = imgs.cuda()
            labels = labels.cuda()
            
            #print(imgs.shape)

            y_pred = model(imgs)

            loss = criterion(y_pred, labels)
            loss.backward()
            
            train_loss += loss.item()
            train_size += y_pred.size(0)
            train_loss_log.append(loss.data / y_pred.size(0))
            
            train_pred += (y_pred.argmax(1) == labels).sum()

            optimizer.step()            

        train_acc_log.append(train_pred / train_size)

        val_loss = 0.
        val_size = 0
        
        val_pred = 0.
        
        model.eval()
        
        delta = np.zeros(30)
        
        with torch.no_grad():
            for imgs, labels in test_dataloader:
                
                #print(imgs.shape)
                
                imgs = imgs.cuda()
                labels = labels.cuda()
                
                pred = model(imgs)
                loss = criterion(pred, labels)
                
                val_loss += loss.item()
                val_size += pred.size(0)
                
                indexis = torch.abs(labels - pred.argmax(1)).cpu().detach().numpy() 
                np.add.at(delta, indexis, 1)                
                val_pred += (pred.argmax(1) == labels).sum()
        

        val_loss_log.append(val_loss / val_size)
        val_acc_log.append(val_pred / val_size)

        
        #plot_history(train_loss_log, val_loss_log, 'loss')
        
        clear_output()
        

        print("learning rate:", optimizer.param_groups[0]["lr"])
        print('Train loss:', (train_loss / train_size)*100)
        print('Val loss:', (val_loss / val_size)*100)
        print('Train acc:', (train_pred / train_size)*100)
        print('Val acc:', (val_pred / val_size)*100)
        print('D: ', delta)
                        
        
        d = []
        for i in range(6):
            if i!=0:
                d.append(d[len(d)-1]+delta[i])
            else:
                d.append(delta[i])
                
        d = d/delta.sum()
        
        print('===ERR===\n        c          b')
        
        for i in range(len(d)):
            if best_model_attr:
                print(i, d[i], best_model_attr[2][i])
            else:    
                print(i, d[i])
        
        r2_surg = sum([ i*i *delta[i] for i in range(len(delta))])/1000.
        
        print("SUROGAT-R2:", r2_surg)
        if best_model_attr:
            print("SUROGAT-R2_best:", best_model_attr[0])
        
        scheduler.step()
        
        if best_model_attr is None or best_model_attr[0] > r2_surg:
            best_model_attr = (r2_surg, copy.deepcopy(model), d)
        
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log, best_model_attr

In [7]:
def valid(model, valid_loader):
    
    model.eval()
    valid_predicts = []

    for imgs, label in valid_loader:
        
        imgs = imgs.cuda()
        pred = model(imgs)

        pred_numpy = pred.cpu().detach().numpy()

        for class_obj in pred_numpy:
            index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])            
            valid_predicts.append(index)
        
    return valid_predicts

In [8]:
def run_fit_model(train_loader, valid_loader, lr, epoch, momentum, milestone, gamma, cycle, gamma_pow, weight_decay):

    torch.cuda.empty_cache()
    torch.manual_seed(0)

    model = models.resnet50(pretrained=pretrained)
    model.fc = nn.Linear(2048, 25)

    model = model.cuda()

    criterion = torch.nn.CrossEntropyLoss()

    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum, weight_decay = weight_decay)

    scheduler_instance = scheduler_custom_func (milestone, gamma, cycle, gamma_pow)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = scheduler_instance)

    train_loss_log, train_acc_log, val_loss_log, val_acc_log, best_model_attr = train(model, 
                                                                 criterion, 
                                                                 optimizer, 
                                                                 scheduler,
                                                                 train_loader, 
                                                                 valid_loader, 
                                                                 epoch)
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log, best_model_attr , model

In [9]:
def valid_models(models, valid_loader, file_log):
    r2_vals = []
    for ind_m in range(len(models)):
    
        torch.cuda.empty_cache()
        model = models[ind_m]
        model.to("cuda:0")
    
        valid_predicts = valid(model, valid_loader)
        valid_df["pred"] = valid_predicts
        valid_df['diff'] = valid_df['number_of_houses']-valid_df['pred']

        r2 = metrics.r2_score(valid_df['number_of_houses'].values, valid_df['pred'].values)
        acc = metrics.accuracy_score(valid_df['number_of_houses'].values, valid_df['pred'].values)
        hist_err = valid_df['diff'].value_counts()/valid_df.shape[0]
        log_val = f"\nmodel {ind_m:d}"
        log_val = log_val + f"\nr2: {r2:.4f}\nacc: {acc:.4f}"
    
        log_val = log_val + "\n=====errors===="
        d = []
        for i in range(6):
            if i!=0:
                c = d[len(d)-1]
                if i in hist_err.index:
                    c = c + hist_err[i]
                if -i in hist_err.index:
                    c = c + hist_err[-i]
                d.append(c)
            else:
                d.append(hist_err[i])
        for i in range(len(d)):
            log_val = log_val + f"\n{i:d} {d[i]:.4f}"
        if file_log:
            file_log.write(log_val)
        else:
            print(log_val)
        r2_vals.append(r2)
    return r2_vals

In [10]:
def test_builder(test_loader):
        

    def test_1(model, data_loader):
    
        model.eval()
        predicts = []

        for imgs in data_loader:
        
            imgs = imgs.cuda()
            pred = model(imgs)

            pred_numpy = pred.cpu().detach().numpy()

            for class_obj in pred_numpy:
                index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])            
                predicts.append(index)
        
        return predicts

    

    def run_test(model, r2):
        predicts = test_1(model, test_loader)


        data_1 = [(name, pred) for name, pred in zip(test_df["img_num"].values.tolist(), predicts)]
        submit_df = pd.DataFrame(data_1, columns=['img_num', 'number_of_houses'])

        #Не забываем добавить 1 к предсказываемому лейблу
        submit_df.number_of_houses = submit_df.number_of_houses + 1

        submit_df.to_csv(f"results_log/submit-{r2:.4f}.csv", index=False)
        print("DONE!!!")
        
    return run_test

In [11]:
def scheduler_custom_func(milestone, gamma=0.5, cycle = 10, gamma_pow = None):
    def calc_lr(epoch):
        if cycle:
            e = epoch % cycle +1        
        else:
            e = epoch
        e1 = 1
        for v in milestone:
            if v <= e:
                e1 = e1*gamma        
        if gamma_pow:
            for v in gamma_pow:
                if v <= epoch:
                    e1 = e1*gamma
        return e1
    return calc_lr

In [12]:
# читаем датасет
data_df = pd.read_csv(base_path+f"/{train_alias}.csv")
# Нам придется вычесть 1 что индексы классов начинались с 0.
data_df.number_of_houses = data_df.number_of_houses- 1

data_df = data_df[data_df.number_of_houses < 25]


In [13]:
import random

def custom_dataset_split(data_df, test_size, random_state):
    random.seed(random_state)
    np.random.seed(random_state)

    valid_lst = []
    train_lst = []

    part_valid = test_size

    flag_train = False

    for i in range(len(data_df)):
        image_name, label = data_df.iloc[i]['img_num'], data_df.iloc[i]['number_of_houses']    
        if not("_" in image_name):
            r = random.random()
            if r <= part_valid:
                flag_train = False
                valid_lst.append({'img_num':image_name, 'number_of_houses':label})
            else:
                flag_train = True
                train_lst.append({'img_num':image_name, 'number_of_houses':label})
        else:
            if flag_train:
                train_lst.append({'img_num':image_name, 'number_of_houses':label})

    print(len(data_df),":" ,len(train_lst),"+",len(valid_lst),'x11 =',len(train_lst)+len(valid_lst)*11)  

    train_df = pd.DataFrame(train_lst, columns = ['img_num','number_of_houses'])
    #print(train_df.head(5))
    train_df = train_df.reindex(np.random.permutation(train_df.index))
    #print(train_df.head(5))
    valid_df = pd.DataFrame(valid_lst, columns = ['img_num','number_of_houses'])
    return train_df, valid_df

# разделим датасет на трейн и валидацию, чтобы смотреть на качество
if train_alias == 'train':
    train_df, valid_df = train_test_split(data_df, test_size=0.2, random_state=43)
if train_alias == 'train_a':
    train_df, valid_df = custom_dataset_split(data_df, test_size=0.2, random_state=43)

In [14]:
g = torch.Generator()
g.manual_seed(initial_seed)

if (train_alias == 'train'):
    train_dataset = ImageDataset(train_df, train_alias, transform_train)
else:
    train_dataset = ImageDataset(train_df, train_alias, transform_valid)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=36,
                                           shuffle=True,
                                           pin_memory=True,                                           
                                           num_workers=num_workers,
                                           worker_init_fn=seed_worker,
                                           generator=g
                                          )


valid_dataset = ImageDataset(valid_df, train_alias ,transform_valid)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=36,
                                           # shuffle=True,
                                           pin_memory=True,
                                           num_workers=num_workers,                                           )

test_dataset = TestImageDataset(test_df, transform_valid)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=1,
                                           # shuffle=True,
                                           pin_memory = True,
                                           num_workers = num_workers)

In [15]:
gamma = 0.5

lr_list = [0.01]
epoch_lst = [40]
momentum_lst = [0.93]
milestone_lst = [[2,4,8,16,20, 30]]
gamma_pow_lst = [None]
cycle_lst = [None] 
weight_decay=0


fp = open('results_log/result_log', 'w')
fp.write("\n")
fp.close()

best_r2 = -10;

for lr in lr_list:
    for epoch in epoch_lst:
        for momentum in momentum_lst:
            for milestone in milestone_lst:
                for gamma_pow in gamma_pow_lst:
                    for cycle in cycle_lst:
                        
                        seed_worker(0)
                        
                        train_loss_log, train_acc_log, val_loss_log, val_acc_log, best_model_attr , model = run_fit_model(train_loader, valid_loader, lr, epoch, momentum, milestone, gamma, cycle, gamma_pow, weight_decay = weight_decay)
                        
                        fp = open('results_log/result_log', 'a')
                        fp.write("\nepoch: "+str(epoch)+"  momentum:"+str(momentum)+"  milestone:"+str(milestone)+" gamma_pow:"+str(gamma_pow)+" cycle:" + str(cycle))
                        
                        lst_models = []
                        best_model = best_model_attr[1]
                        lst_models.append(best_model)
                        
                        curr_model = model.to('cpu')
                        lst_models.append(curr_model)
                        
                        r2_vals = valid_models(lst_models, valid_loader, fp)                                            
                                                                    
                        fp.close()
                        
                        for i in range(len(lst_models)):
                            if r2_vals[i] > best_r2:
                                torch.cuda.empty_cache()
                                model_1 = lst_models[i]
                                model_1.to("cuda:0")
                                execute_test = test_builder(test_loader)
                                execute_test(model_1, r2_vals[i])
                                best_r2 = r2_vals[i]
                                torch.save(model_1, f'models_log/model-{best_r2:.4f}.pth')
                                

100%|██████████| 40/40 [10:46<00:00, 16.15s/it]

learning rate: 0.00015625
Train loss: 2.076061317873769
Val loss: 6.888976608003889
Train acc: tensor(79.4275, device='cuda:0')
Val acc: tensor(30.0000, device='cuda:0')
D:  [126. 107.  83.  51.  28.  13.   4.   4.   2.   1.   1.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.]
===ERR===
        c          b
0 0.3 0.2785714285714286
1 0.5547619047619048 0.611904761904762
2 0.7523809523809524 0.7976190476190477
3 0.8738095238095238 0.8952380952380953
4 0.9404761904761905 0.9476190476190476
5 0.9714285714285714 0.9714285714285714
SUROGAT-R2: 2.32
SUROGAT-R2_best: 2.118



/home/user/torch111/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/user/torch111/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/user/torch111/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

DONE!!!


In [16]:

#print("r2_srg:", best_model_attr[0])

#lst_models = []
#best_model = best_model_attr[1]
#lst_models.append(best_model)

#torch.save(best_model, 'best_model.pth')

#curr_model = model.to('cpu')
#lst_models.append(curr_model)


In [17]:
#torch.cuda.empty_cache()
#model = best_model
#model.to("cuda:0")